# Make Pixel Intensity Histograms of your favorite pictures!
A video that fits well with our analysis method has a peak for the whites and the blacks with a dip inbetween


You only need to run the cell below this one once to import all the functions and modules. The only packages the code needs is numpy and matplotlib. You might need to install these if you haven't already. If you haven't run the installation blocks to install

In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.path import Path
# from pathlib import Path
from matplotlib.widgets import Cursor
import os
import cv2

%matplotlib qt
def onclick(event):
    """
    Plot a polygon where the user clicks
    """
    x, y = event.xdata, event.ydata
    clicked_points.append([x, y])
    xy = np.asarray(clicked_points)
    points_plot.set_data(xy[:, 0], xy[:, 1])
    if len(xy) < 2:
        return
    polygon_plot.set_data(np.append(xy[:, 0], xy[0, 0]), np.append(xy[:, 1],xy[0, 1])) #[xy[0, 1]] +[xy[0, 0]]
    plt.draw()
    return

def final_polygon(event):
    """check which points are inside the polygon when the user presses 'enter'
    The points inside the selection are deleted from the ROI
    """
    global pix_i, pix_j, image
    if event.key == 'enter':
        image_height, image_width = image.shape[:2]
        path0 = Path(np.asarray(clicked_points))
        x0, y0 = np.meshgrid(np.arange(image_width), np.arange(image_height))
        points0 = np.vstack((x0.ravel(), y0.ravel())).T
        mask = path0.contains_points(points0).reshape((image_height, image_width))
        fig.local_pixel_intensity = image[mask]
        chosen_pixels.set_data(x0[mask], y0[mask])
        plt.draw()
        return

def generate_histogram(local_pixel_intensity, bins=256, range=(0, 256)):
    """generate a histogram of the pixel intensities"""
    fig, ax = plt.subplots()
    ax.hist(local_pixel_intensity, bins=256, range=(0, 256))
    ax.set_xlabel('Pixel Intensity')
    ax.set_ylabel('Frequency')
    plt.show()

In [ ]:
# Only run this cell if you have not installed numpy or matplotlib yet!
# %pip install numpy
# %pip install matplotlib
# %pip install opencv-python    #For saving the frame in the video as an image

# Save a frame from a video
The Histogram is generated for a still frame. If you have a video, first svae the frame of interest as a picture with the module below

In [ ]:

video_root = "C:\\Users\\thijs\\Documents\\GitHub\\pyidi\\Blackledge_media\\"
video_path = os.path.normpath(video_root + "Ar.tri.14_11.15.22_30mm_natfq(full).mp4")
if not os.path.exists(video_path):
    print(f"Video file '{video_path}' does not exist")
if not os.access(video_path, os.R_OK):
    print(f"Cannot read video file '{video_path}'")
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Cannot open video file '{video_path}'")
flag = cv2.IMREAD_GRAYSCALE
ret, frame = cap.read()
if ret:
    # Do something with the frame, such as convert it to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # save the frame
    extension = 'jpg'
    cv2.imwrite(video_root + f'frame.{extension}', gray)
cap.release()

# Load your image in the cell bewow. Replace the path to a picture of your choice 

In [3]:
video_root = "C:\\Users\\thijs\\Documents\\GitHub\\pyidi\\Blackledge_media\\"
extension = 'jpg'
picture_path = os.path.normpath(video_root + f'frame.{extension}')
image = mpimg.imread(picture_path)

In [4]:
clicked_points = []
fig, ax = plt.subplots(figsize=(15, 5))
ax.set_title('Keep adding points to your Polygon, press enter when done')
points_plot, = plt.plot([], [], 'b.')
polygon_plot, = plt.plot([], [], 'b-', linewidth=1)
chosen_pixels, = plt.plot([], [], 'r', linewidth=1)
ax.imshow(image, cmap='gray')
ax.set_xlim(-10, image.shape[1]+10)
ax.set_ylim(image.shape[0]+10, -10)
cursor = Cursor(ax, useblit=True, color='blue', linewidth=1)
fig.canvas.mpl_connect('button_press_event', onclick)
fig.canvas.mpl_connect('key_press_event', final_polygon)

Click on the image to select the ROI


Finally, generate your histogram by running the cell below

In [5]:
local_pixel_intensity = fig.local_pixel_intensity
generate_histogram(local_pixel_intensity)

Don't want to generate a histogram of a selection, but of the full picture? Run the cell below

In [6]:
generate_histogram(image.flatten())